Start recordings at the same time.

In [27]:
from pupil_labs.realtime_api.simple import discover_devices, discover_one_device, Device
from datetime import datetime
import nest_asyncio
import threading
import time

nest_asyncio.apply()

In [28]:
# devices = discover_devices(search_duration_seconds=10.0)
devices = discover_one_device(max_search_duration_seconds=10.0)                                                                                   
devices                           

Device(ip=10.206.114.147, port=8080, dns=neon-1.local.)

In [17]:
ip = "10.206.114.147"
device = Device(address=ip, port="8080")

In [26]:
# Look for devices. Returns as soon as it has found the first device.
print("Looking for the next best device...")

# Device status is fetched on initialization and kept up-to-date in the background
for device in [devices]:

    if device is None:
        print("No device found.")
        raise SystemExit(-1)

    print(f"Phone IP address: {device.phone_ip}")
    print(f"Phone name: {device.phone_name}")
    print(f"Phone unique ID: {device.phone_id}")

    print(f"Battery level: {device.battery_level_percent}%")
    print(f"Battery state: {device.battery_state}")

    print(f"Free storage: {device.memory_num_free_bytes / 1024**3}GB")
    print(f"Storage level: {device.memory_state}")

    print(f"Connected glasses: SN {device.serial_number_glasses}")
    print(f"Connected scene camera: SN {device.serial_number_scene_cam}")
    print()

    device.close()  # explicitly stop auto-update

Exception ignored in: <async_generator object connect.__aiter__ at 0x12ce463e0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/eyetracker/lib/python3.13/site-packages/pupil_labs/realtime_api/device.py", line 82, in status_updates
    break
RuntimeError: async generator ignored GeneratorExit


Looking for the next best device...
Phone IP address: 10.206.114.147
Phone name: IRIS
Phone unique ID: 6a304536c7211443
Battery level: 38%
Battery state: OK
Free storage: 214.16532516479492GB
Storage level: OK
Connected glasses: SN -1
Connected scene camera: SN -1



In [82]:
barrier = threading.Barrier(2)  # Ensures both threads start together


In [85]:
def start_single_recording(device, duration, barrier): 
    barrier.wait() 
    print(f"Starting recording for {device.phone_name}")
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("Current timestamp:", timestamp)
    recording_id = device.recording_start()
    print(f"Started recording with id {recording_id}")

    time.sleep(duration)

    device.recording_stop_and_save()

    print("Recording stopped and saved")
    print("End timestamp", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    # device.recording_cancel()  # uncomment to cancel recording
    device.close()

In [86]:
threading.Thread(target=start_single_recording, args=(devices[0], 10, barrier)).start()
threading.Thread(target=start_single_recording, args=(devices[1], 10, barrier)).start()

Starting recording for THEIAStarting recording for IRIS
Current timestamp: 2025-02-20 17:14:59

Current timestamp: 2025-02-20 17:14:59


Started recording with id 18ecd1aa-aa84-4601-8478-67a92c2c25f3
Started recording with id 9c467f7e-fbba-4ea7-b834-90f6c9419137
Recording stopped and saved
End timestamp 2025-02-20 17:15:09
Recording stopped and saved
End timestamp 2025-02-20 17:15:09


In [19]:
from pupil_labs.realtime_api.simple import discover_one_device

 
if device is None:
    print("No device found.")
    raise SystemExit(-1)

print(f"Starting recording")
recording_id = device.recording_start()
print(f"Started recording with id {recording_id}")

time.sleep(12)

device.recording_stop_and_save()

print("Recording stopped and saved")
# device.recording_cancel()  # uncomment to cancel recording

device.close()

Starting recording
Started recording with id 05d0bd6f-9ade-4e1a-8ed9-9499a90fff88
Recording stopped and saved


In [7]:
estimate = device.estimate_time_offset()
if estimate is None:
    device.close()
    raise SystemExit("Pupil Companion app is too old")

print(f"Mean time offset: {estimate.time_offset_ms.mean} ms")
print(f"Mean roundtrip duration: {estimate.roundtrip_duration_ms.mean} ms")

Mean time offset: 17.3 ms
Mean roundtrip duration: 32.89 ms
